In [1]:
import opal
import kglab
import rdflib
import z3
import dateutil.parser
import re
import numpy as np
import pandas as pd
import os
import sexpdata

In [2]:
from opal.logic.z3.z3_literal import Z3Literal
from opal.data.event_data import EventData
from opal.data.mapper import LogMapper
from opal.data.processor import LogProcessor

import opal.logic.z3.z3_literal as z3_literal
from opal.logic.z3.z3_ontology import Z3Ontology as Ontology
from opal.logic.z3.z3_reasoner import Z3Reasoner as Reasoner

from rdflib import Graph

from z3 import *
z3.set_param(proof=True)

Adding axiom: PSL Core Ontology_ont_t_0_type_disjointness
Adding axiom: PSL Core Ontology_ont_t_1_begin_unique
Adding axiom: PSL Core Ontology_ont_t_2_ends_unique
Adding axiom: PSL Core Ontology_ont_t_3_occurrence_bounds
Adding axiom: PSL Core Ontology_ont_t_4_occurrence_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_5_unique_activity_occurrence
Adding axiom: PSL Core Ontology_ont_t_6_occurrence_has_activity
Adding axiom: PSL Core Ontology_ont_t_7_participation_t_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_8_participation_time_to_generic
Adding axiom: PSL Core Ontology_ont_t_9_participation_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_10_participation_temporal_constraint
Adding axiom: PSL Core Ontology_ont_t_11_object_temporal_existence
Adding axiom: PSL Core Ontology_ont_t_12_occurrence_temporal_extent
Adding axiom: PSL Core Ontology_map_t_0_occurrence_start_end_event_mapping
Adding axiom: PSL Core Ontology_map_t_1_occurrence_participation_mapping
Adding a

In [3]:
data_dir = os.path.abspath('../data/')
ttl_data_file = data_dir + '/P1_log_instances.ttl'
sample_log_file = data_dir + '/sample_log.csv'
bpic2017_file = data_dir + '/BPI Challenge 2017.xes.gz'

ontology_dir = os.path.abspath('../../opal/logic/z3/ontologies/')
psl_core_file = ontology_dir + '/PSL/PSL_core.smt2'
psl_core_mapping_file = ontology_dir + '/PSL/PSL_core_mapping.smt2'

## Mapping simple XES data into logical facts

In [4]:
# load the log
col_dict = {'caseID' : 'caseID', 'activityID' : 'activityID', 'timestamp' : 'timestamp', 'eventID' : None, 'transition' : 'transition'}
processor = LogProcessor(col_dict=col_dict, log_path=sample_log_file, batch_size=1)
event_data = processor.load_log()

In [5]:
event_data.data

,caseID,activityID,eventID,timestamp,resourceID,transition,processID,batch
0,C_case_0,A_activity_A,E_0,2016-01-01 09:00:00+00:00,user_1,T_start,P1,1
1,C_case_0,A_activity_A,E_1,2016-01-01 09:15:00+00:00,user_1,T_complete,P1,1
2,C_case_0,A_activity_C,E_2,2016-01-01 09:35:00+00:00,user_1,T_complete,P1,1
3,C_case_1,A_activity_A,E_3,2016-01-02 09:00:00+00:00,user_1,T_complete,P1,2
4,C_case_1,A_activity_B,E_4,2016-01-02 09:00:00+00:00,user_0,T_complete,P1,2
5,C_case_1,A_activity_C,E_5,2016-01-03 09:00:00+00:00,user_0,T_complete,P1,2
6,C_case_2,A_activity_A,E_6,2016-01-01 09:00:00+00:00,user_1,T_complete,P1,3
7,C_case_2,A_activity_B,E_7,2016-01-01 09:10:00+00:00,user_1,T_complete,P1,3
8,C_case_2,A_activity_C,E_8,2016-01-01 09:15:00+00:00,user_2,T_complete,P1,3
9,C_case_2,A_activity_D,E_9,2017-01-06 10:35:00+00:00,user_1,T_complete,P1,3


In [6]:
event_data.metadata

{'log_paths': ['/tmp/tmpf203zs3p.csv',
  '/tmp/tmpgic4fpbq.csv',
  '/tmp/tmpks9_lsv0.csv'],
 'process_name': 'P1',
 'batches': 3}

In [7]:
mapper = LogMapper(data=event_data, output_encoding='z3')
z3_data = mapper.get_mapped_data()

Building mappings for logs...


  0%|          | 0/3 [00:00<?, ?it/s]2025-10-09 00:07:55,318 | INFO: Translating YARRRML mapping to [R2]RML
2025-10-09 00:07:55,319 | INFO: RML content is created!
2025-10-09 00:07:55,326 | INFO: Mapping has been syntactically validated.
2025-10-09 00:07:55,327 | INFO: Translation has finished successfully.
2025-10-09 00:07:55,334 | INFO: Translating YARRRML mapping to [R2]RML
2025-10-09 00:07:55,336 | INFO: RML content is created!
2025-10-09 00:07:55,342 | INFO: Mapping has been syntactically validated.
2025-10-09 00:07:55,343 | INFO: Translation has finished successfully.
2025-10-09 00:07:55,349 | INFO: Translating YARRRML mapping to [R2]RML
2025-10-09 00:07:55,350 | INFO: RML content is created!
2025-10-09 00:07:55,357 | INFO: Mapping has been syntactically validated.
2025-10-09 00:07:55,357 | INFO: Translation has finished successfully.
100%|██████████| 3/3 [00:00<00:00, 59.25it/s]

Mappings built.
Generating knowledge graph...



INFO | 2025-10-09 00:07:55,879 | 11 mapping rules retrieved.
INFO | 2025-10-09 00:07:55,892 | Mapping partition with 11 groups generated.
INFO | 2025-10-09 00:07:55,893 | Maximum number of rules within mapping group: 1.
INFO | 2025-10-09 00:07:55,894 | Mappings processed in 0.530 seconds.
INFO | 2025-10-09 00:07:56,274 | Number of triples generated in total: 25.


Knowledge graph generated.
Knowledge graph 1/3 generated.
Generating knowledge graph...


INFO | 2025-10-09 00:07:56,776 | 11 mapping rules retrieved.
INFO | 2025-10-09 00:07:56,788 | Mapping partition with 11 groups generated.
INFO | 2025-10-09 00:07:56,789 | Maximum number of rules within mapping group: 1.
INFO | 2025-10-09 00:07:56,789 | Mappings processed in 0.506 seconds.
INFO | 2025-10-09 00:07:57,143 | Number of triples generated in total: 26.


Knowledge graph generated.
Knowledge graph 2/3 generated.
Generating knowledge graph...


INFO | 2025-10-09 00:07:57,640 | 11 mapping rules retrieved.
INFO | 2025-10-09 00:07:57,652 | Mapping partition with 11 groups generated.
INFO | 2025-10-09 00:07:57,653 | Maximum number of rules within mapping group: 1.
INFO | 2025-10-09 00:07:57,653 | Mappings processed in 0.503 seconds.
INFO | 2025-10-09 00:07:57,988 | Number of triples generated in total: 33.


Knowledge graph generated.
Knowledge graph 3/3 generated.
Z3 literal set generated (1/3)
Z3 literal set generated (2/3)
Z3 literal set generated (3/3)


In [8]:
# rdf is still generated, because it is a prerequisite for mapping to more expressive encoding languages
graph = mapper.kgs[0].rdf_graph()
for triple in graph.triples((None, None, None)):
    print(triple)

(rdflib.term.URIRef('http://www.example.com/E_0'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasCase'), rdflib.term.URIRef('http://www.example.com/C_case_0'))
(rdflib.term.URIRef('http://www.example.com/E_1'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasResource'), rdflib.term.URIRef('http://www.example.com/user_1'))
(rdflib.term.URIRef('http://www.example.com/E_1'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasCase'), rdflib.term.URIRef('http://www.example.com/C_case_0'))
(rdflib.term.URIRef('http://www.example.com/C_case_0'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasProcess'), rdflib.term.URIRef('http://www.example.com/P1'))
(rdflib.term.URIRef('http://www.example.com/E_1'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasTransition'), rdflib.term.URIRef('http://www.example.com/T_complete'))
(rdflib.term.URIRef('http://www.example.com/C_case_0'), rdflib.term.URIRef('http://www.w

In [9]:
from opal.logic.z3.z3_handler import PSL_CORE_MAPPING_ENV
for literal in z3_data:
    expr = literal.z3_expr
    print(expr)
    
# get a random expression to check if contexts remain the same with additional ontologies loaded
random_idx = np.random.randint(len(z3_data))
random_expr = z3_data[random_idx].z3_expr
print(f'Matching contexts: {random_expr.ctx == PSL_CORE_MAPPING_ENV["ctx"]}')

hasCase(E_0, C_case_0)
hasResource(E_1, user_1)
hasCase(E_1, C_case_0)
hasProcess(C_case_0, P1)
hasTransition(E_1, T_complete)
Case(C_case_0)
Activity(A_activity_A)
hasTransition(E_0, T_start)
Activity(A_activity_C)
Transition(T_start)
hasActivity(E_0, A_activity_A)
hasRecordedTime(E_2, 1451640900)
hasActivity(E_1, A_activity_A)
Event(E_1)
Event(E_2)
hasTransition(E_2, T_complete)
Event(E_0)
hasResource(E_0, user_1)
Transition(T_complete)
hasRecordedTime(E_1, 1451639700)
Resource(user_1)
hasResource(E_2, user_1)
hasRecordedTime(E_0, 1451638800)
hasCase(E_2, C_case_0)
hasActivity(E_2, A_activity_C)
Event(E_3)
hasRecordedTime(E_4, 1451725200)
hasProcess(C_case_1, P1)
Case(C_case_1)
hasResource(E_3, user_1)
Activity(A_activity_A)
hasTransition(E_3, T_complete)
Activity(A_activity_B)
hasActivity(E_4, A_activity_B)
hasActivity(E_5, A_activity_C)
hasCase(E_4, C_case_1)
hasRecordedTime(E_3, 1451725200)
Activity(A_activity_C)
Event(E_4)
hasCase(E_5, C_case_1)
hasCase(E_3, C_case_1)
Event(E_5)


In [10]:
PSL_CORE_MAPPING_ENV['functions'] 

{'Event': Event,
 'Transition': Transition,
 'Activity': Activity,
 'Case': Case,
 'Resource': Resource,
 'hasResource': hasResource,
 'hasRecordedTime': hasRecordedTime,
 'hasLifecycleTransition': hasLifecycleTransition,
 'hasCase': hasCase,
 'hasActivity': hasActivity,
 'activity': activity,
 'activity_occurrence': activity_occurrence,
 'object_': object_,
 'timepoint': timepoint,
 'occurrence_of': occurrence_of,
 'participates_in': participates_in,
 'participates': participates,
 'exists_at': exists_at,
 'is_occurring_at': is_occurring_at,
 'begin_of': begin_of,
 'end_of': end_of,
 'arboreal': arboreal,
 'earlier': earlier,
 'earlierEq': earlierEq,
 'initial': initial,
 'generator': generator,
 'legal': legal,
 'precedes': precedes,
 'poss': poss,
 'successor': successor,
 'subactivity': subactivity,
 'primitive': primitive,
 'atomic': atomic,
 'conc': conc,
 'hand_off': hand_off,
 'ping_pong': ping_pong,
 'min_precedes': min_precedes,
 'root': root,
 'subtree': subtree,
 'next_subo

### Reasoning Capabilities

In [11]:
# load the ontology (together with the mapping)
psl_core = Ontology.from_smt2(psl_core_file, psl_core_mapping_file)

Adding axiom: None_ont_t_0_type_disjointness
Adding axiom: None_ont_t_1_begin_unique
Adding axiom: None_ont_t_2_ends_unique
Adding axiom: None_ont_t_3_occurrence_bounds
Adding axiom: None_ont_t_4_occurrence_sort_constraints
Adding axiom: None_ont_t_5_unique_activity_occurrence
Adding axiom: None_ont_t_6_occurrence_has_activity
Adding axiom: None_ont_t_7_participation_t_sort_constraints
Adding axiom: None_ont_t_8_participation_time_to_generic
Adding axiom: None_ont_t_9_participation_sort_constraints
Adding axiom: None_ont_t_10_participation_temporal_constraint
Adding axiom: None_ont_t_11_object_temporal_existence
Adding axiom: None_ont_t_12_occurrence_temporal_extent
Adding axiom: None_map_t_0_occurrence_start_end_event_mapping
Adding axiom: None_map_t_1_occurrence_participation_mapping


In [12]:
psl_core.axioms

[{'name': 'None_ont_t_0_type_disjointness',
  'description': 'Activities, occurrences, and objects are different things.',
  'expr': '(assert (! (forall ((x Ind)) (and (=> (activity x) (not (or (activity_occurrence x) (object_ x)))) (=> (activity_occurrence x) (not (or (object_ x) (activity x)))) (=> (object_ x) (not (or (activity_occurrence x) (activity x)))))) :named type_disjointness :description "Activities, occurrences, and objects are different things."))'},
 {'name': 'None_ont_t_1_begin_unique',
  'description': 'Start points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (begin_of x) t1) (= (begin_of x) t2)) (= t1 t2))) :named begin_unique :description "Start points are unique."))'},
 {'name': 'None_ont_t_2_ends_unique',
  'description': 'End points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (end_of x) t1) (= (end_of x) t2)) (= t1 t2))) :named ends_unique :description "End points are unique."))'},

In [13]:
psl_core.literals

[]

In [14]:
z3_facts = []
for i, fact in enumerate(z3_data):
    prefix = f'mapped_a_{i}_'
    expr = fact
    name = prefix + f'{expr}'
    z3_facts.append({name : expr})
    

In [15]:
from opal.logic.z3.z3_ontology import PSL_CORE_Z3, PSL_OCCTREE_Z3, PSL_COMPLEX_Z3, PSL_SUBACTIVITY_Z3, PSL_ATOMIC_Z3
print(PSL_CORE_Z3.name)
print(PSL_OCCTREE_Z3.name)
print(PSL_COMPLEX_Z3.name)

PSL Core Ontology
PSL Occtree Ontology
PSL Complex Ontology


In [16]:
PSL_COMPLEX_Z3.axioms

[{'name': 'PSL Core Ontology_ont_t_0_type_disjointness',
  'description': 'Activities, occurrences, and objects are different things.',
  'expr': '(assert (! (forall ((x Ind)) (and (=> (activity x) (not (or (activity_occurrence x) (object_ x)))) (=> (activity_occurrence x) (not (or (object_ x) (activity x)))) (=> (object_ x) (not (or (activity_occurrence x) (activity x)))))) :named type_disjointness :description "Activities, occurrences, and objects are different things."))'},
 {'name': 'PSL Core Ontology_ont_t_1_begin_unique',
  'description': 'Start points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (begin_of x) t1) (= (begin_of x) t2)) (= t1 t2))) :named begin_unique :description "Start points are unique."))'},
 {'name': 'PSL Core Ontology_ont_t_2_ends_unique',
  'description': 'End points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (end_of x) t1) (= (end_of x) t2)) (= t1 t2))) :named ends_unique :de

In [20]:
r = Reasoner()
r.load_ontology(PSL_SUBACTIVITY_Z3)
r.load_facts(z3_facts)
r.add_domain_closure_axioms('Event')

Adding domain closure axiom for class Event


In [18]:
s = r.check()

In [19]:
# Since the data is consistent with the ontology, we have a satisfying interpretation and can extract a model
print(s.model())

[mapped_a_9_Transition(T_start) = True,
 PSL Core Ontology_ont_t_6_occurrence_has_activity = True,
 mapped_a_43_Transition(T_complete) = True,
 A_activity_A = Ind!val!6,
 E_8 = Ind!val!18,
 mapped_a_49_hasResource(E_5, user_0) = True,
 mapped_a_56_hasResource(E_8, user_2) = True,
 mapped_a_63_hasTransition(E_7, T_complete) = True,
 mapped_a_7_hasTransition(E_0, T_start) = True,
 E_0 = Ind!val!0,
 mapped_a_36_hasRecordedTime(E_3, 1451725200) = True,
 mapped_a_50_hasTransition(E_5, T_complete) = True,
 mapped_a_5_Case(C_case_0) = True,
 mapped_a_37_Activity(A_activity_C) = True,
 mapped_a_31_hasTransition(E_3, T_complete) = True,
 mapped_a_70_hasRecordedTime(E_6, 1451638800) = True,
 mapped_a_58_hasCase(E_6, C_case_2) = True,
 mapped_a_1_hasResource(E_1, user_1) = True,
 E_7 = Ind!val!16,
 PSL Core Ontology_ont_t_7_participation_t_sort_constraints = True,
 mapped_a_21_hasResource(E_2, user_1) = True,
 mapped_a_45_hasResource(E_4, user_0) = True,
 mapped_a_54_Activity(A_activity_B) = True

In [20]:
from opal.logic.z3.z3_handler import REFERENCE_TAXONOMY_ENV
REFERENCE_TAXONOMY_ENV['constants']

{'T_start': T_start,
 'T_complete': T_complete,
 'E_1': E_1,
 'user_1': user_1,
 'E_2': E_2,
 'E_0': E_0,
 'C_case_0': C_case_0,
 'A_activity_A': A_activity_A,
 'A_activity_C': A_activity_C,
 'P1': P1,
 'A_activity_B': A_activity_B,
 'user_0': user_0,
 'E_4': E_4,
 'E_5': E_5,
 'E_3': E_3,
 'C_case_1': C_case_1,
 'E_8': E_8,
 'E_6': E_6,
 'E_7': E_7,
 'user_2': user_2,
 'E_9': E_9,
 'C_case_2': C_case_2,
 'A_activity_D': A_activity_D}

In [20]:
r.functions

{'Event': Event,
 'Transition': Transition,
 'Activity': Activity,
 'Case': Case,
 'Resource': Resource,
 'hasResource': hasResource,
 'hasRecordedTime': hasRecordedTime,
 'hasLifecycleTransition': hasLifecycleTransition,
 'hasCase': hasCase,
 'hasActivity': hasActivity,
 'activity': activity,
 'activity_occurrence': activity_occurrence,
 'object_': object_,
 'timepoint': timepoint,
 'occurrence_of': occurrence_of,
 'participates_in': participates_in,
 'participates': participates,
 'exists_at': exists_at,
 'is_occurring_at': is_occurring_at,
 'begin_of': begin_of,
 'end_of': end_of,
 'arboreal': arboreal,
 'earlier': earlier,
 'earlierEq': earlierEq,
 'initial': initial,
 'generator': generator,
 'legal': legal,
 'precedes': precedes,
 'poss': poss,
 'successor': successor,
 'subactivity': subactivity,
 'primitive': primitive,
 'atomic': atomic,
 'conc': conc,
 'hand_off': hand_off,
 'ping_pong': ping_pong,
 'min_precedes': min_precedes,
 'root': root,
 'subtree': subtree,
 'next_subo

### Pattern Usage

In [21]:
from opal.logic.z3.z3_pattern import HAND_OFF_PATTERN, PING_PONG_PATTERN, PING_PONG_QUERY_PATTERN
ping_pong_q = PING_PONG_QUERY_PATTERN.apply()
s = r.query(ping_pong_q)

Adding query: p_Hand-off_ : 
(assert (! (forall ((r1 Ind) (r2 Ind) (o1 Ind) (o2 Ind) (c Ind))
    (=> (and
            (next_subocc o1 o2 c)
            (participates r1 o1)
            (participates r2 o2)
            (distinct r1 r2)
        )
    (hand_off r1 r2 o1 o2 c))
)))
Adding query: p_Ping-pong_ : 
(assert (! (forall ((c Ind))
    (=> (exists ((e1 Ind) (e2 Ind) (e3 Ind) (e4 Ind) (r1 Ind) (r2 Ind) (r3 Ind) (r4 Ind))
            (and
                (hand_off r1 r2 e1 e2 c)
                (hand_off r3 r4 e3 e4 c)
                (distinct e1 e3)
                (distinct e2 e4)
            ))
        (ping_pong c)
    )
)))
Adding query: p_Ping-pong Query_ : 
(assert (! (not (exists ((c Ind))
    (ping_pong c))
)))


In [22]:
m = s.model()

In [23]:
m

[PSL Core Ontology_ont_t_6_occurrence_has_activity = True,
 E_8 = Ind!val!18,
 A_activity_A = Ind!val!3,
 mapped_a_63_hasResource(E_6, user_1) = True,
 mapped_a_65_Resource(user_1) = True,
 mapped_a_45_hasActivity(E_5, A_activity_C) = True,
 mapped_a_47_hasTransition(E_4, T_complete) = True,
 E_0 = Ind!val!2,
 mapped_a_39_hasRecordedTime(E_4, 1451725200) = True,
 mapped_a_57_Event(E_8) = True,
 mapped_a_82_hasRecordedTime(E_7, 1451639400) = True,
 mapped_a_0_hasTransition(E_2, T_complete) = True,
 mapped_a_23_hasActivity(E_1, A_activity_A) = True,
 mapped_a_64_hasActivity(E_8, A_activity_C) = True,
 mapped_a_29_Activity(A_activity_C) = True,
 mapped_a_59_Transition(T_complete) = True,
 mapped_a_15_hasResource(E_0, user_1) = True,
 E_7 = Ind!val!17,
 PSL Core Ontology_ont_t_7_participation_t_sort_constraints = True,
 mapped_a_66_hasTransition(E_6, T_complete) = True,
 mapped_a_27_hasCase(E_5, C_case_1) = True,
 E_1 = Ind!val!4,
 user_0 = Ind!val!14,
 PSL Core Ontology_ont_t_10_participation_temporal_constraint = True,
 T_complete = Ind!val!1,
 mapped_a_1_hasActivity(E_0, A_activity_A) = True,
 mapped_a_42_hasCase(E_3, C_case_1) = True,
 PSL Core Ontology_ont_t_2_ends_unique = True,
 mapped_a_53_hasCase(E_9, C_case_2) = True,
 user_1 = Ind!val!5,
 mapped_a_60_hasRecordedTime(E_6, 1451638800) = True,
 mapped_a_33_hasTransition(E_3, T_complete) = True,
 mapped_a_67_hasCase(E_7, C_case_2) = True,
 mapped_a_3_Activity(A_activity_C) = True,
 PSL Core Ontology_ont_t_11_object_temporal_existence = True,
 mapped_a_22_hasTransition(E_0, T_start) = True,
 PSL Core Ontology_ont_t_1_begin_unique = True,
 mapped_a_8_hasCase(E_1, C_case_0) = True,
 p_Hand-off_ = True,
 mapped_a_69_Event(E_9) = True,
 PSL Core Ontology_ont_t_0_type_disjointness = True,
 mapped_a_20_hasCase(E_0, C_case_0) = True,
 mapped_a_77_hasActivity(E_6, A_activity_A) = True,
 E_6 = Ind!val!19,
 mapped_a_17_Resource(user_1) = True,
 mapped_a_50_hasActivity(E_3, A_activity_A) = True,
 PSL Core Ontology_map_t_1_occurrence_participation_mapping = True,
 E_2 = Ind!val!0,
 mapped_a_28_Event(E_4) = True,
 PSL Subactivity Ontology_ont_t_1_subactivity_reflexive = True,
 mapped_a_78_Activity(A_activity_A) = True,
 E_9 = Ind!val!16,
 PSL Core Ontology_ont_t_3_occurrence_bounds = True,
 mapped_a_79_hasProcess(C_case_2, P1) = True,
 mapped_a_35_hasTransition(E_5, T_complete) = True,
 PSL Core Ontology_ont_t_9_participation_sort_constraints = True,
 mapped_a_75_Activity(A_activity_C) = True,
 mapped_a_25_hasRecordedTime(E_5, 1451811600) = True,
 mapped_a_43_Resource(user_0) = True,
 mapped_a_5_hasResource(E_2, user_1) = True,
 mapped_a_61_Activity(A_activity_D) = True,
 mapped_a_36_hasResource(E_3, user_1) = True,
 PSL Subactivity Ontology_ont_t_4_subactivity_discrete_up = True,
 A_activity_D = Ind!val!20,
 user_2 = Ind!val!21,
 PSL Subactivity Ontology_ont_t_3_subactivity_transitive = True,
 mapped_a_14_hasActivity(E_2, A_activity_C) = True,
 mapped_a_13_Activity(A_activity_A) = True,
 mapped_a_10_Transition(T_start) = True,
 mapped_a_26_Case(C_case_1) = True,
 mapped_a_49_Event(E_5) = True,
 mapped_a_32_hasProcess(C_case_1, P1) = True,
 mapped_a_80_hasRecordedTime(E_8, 1451639700) = True,
 E_3 = Ind!val!13,
 mapped_a_46_Event(E_3) = True,
 mapped_a_2_hasResource(E_1, user_1) = True,
 mapped_a_71_hasCase(E_6, C_case_2) = True,
 mapped_a_38_Transition(T_complete) = True,
 PSL Core Ontology_ont_t_4_occurrence_sort_constraints = True,
 mapped_a_56_hasCase(E_8, C_case_2) = True,
 mapped_a_62_hasRecordedTime(E_9, 1483698900) = True,
 p_Ping-pong Query_ = True,
 mapped_a_11_Event(E_1) = True,
 PSL Core Ontology_ont_t_12_occurrence_temporal_extent = True,
 mapped_a_40_Resource(user_1) = True,
 mapped_a_44_hasResource(E_5, user_0) = True,
 mapped_a_54_hasResource(E_9, user_1) = True,
 mapped_a_58_hasTransition(E_8, T_complete) = True,
 p_Ping-pong_ = True,
 mapped_a_30_Activity(A_activity_B) = True,
 mapped_a_37_Activity(A_activity_A) = True,
 mapped_a_19_hasRecordedTime(E_1, 1451639700) = Tr

In [39]:
PSL_SUBACTIVITY_Z3.env['functions']['Event']

Event

# Mapping BPIC 2017 data

In [12]:
processor = LogProcessor(log_path=bpic2017_file, batch_size=100)
event_data = processor.load_log(downsample_rate=0.05)

/home/vscode/.local/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:82: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 31509/31509 [00:57<00:00, 546.80it/s]
/workspaces/ProMean Lib/opal/data/processor.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['batch'] = (df.groupby('caseID').ngroup() // self.batch_size) + 1


In [13]:
mapper = LogMapper(data=event_data, output_encoding='z3')
z3_data = mapper.get_mapped_data()

Building mappings for logs...


  0%|          | 0/16 [00:00<?, ?it/s]2025-08-11 14:02:34,488 | INFO: Translating YARRRML mapping to [R2]RML
INFO | 2025-08-11 14:02:34,488 | Translating YARRRML mapping to [R2]RML
2025-08-11 14:02:34,490 | INFO: RML content is created!
INFO | 2025-08-11 14:02:34,490 | RML content is created!
2025-08-11 14:02:34,496 | INFO: Mapping has been syntactically validated.
INFO | 2025-08-11 14:02:34,496 | Mapping has been syntactically validated.
2025-08-11 14:02:34,498 | INFO: Translation has finished successfully.
INFO | 2025-08-11 14:02:34,498 | Translation has finished successfully.
2025-08-11 14:02:34,508 | INFO: Translating YARRRML mapping to [R2]RML
INFO | 2025-08-11 14:02:34,508 | Translating YARRRML mapping to [R2]RML
2025-08-11 14:02:34,509 | INFO: RML content is created!
INFO | 2025-08-11 14:02:34,509 | RML content is created!
2025-08-11 14:02:34,516 | INFO: Mapping has been syntactically validated.
INFO | 2025-08-11 14:02:34,516 | Mapping has been syntactically validated.
2025-08-1

Mappings built.
Generating knowledge graph...


INFO | 2025-08-11 14:02:35,287 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:02:35,297 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:02:35,299 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:02:35,299 | Mappings processed in 0.519 seconds.
INFO | 2025-08-11 14:02:36,870 | Number of triples generated in total: 23614.


Knowledge graph generated.
Knowledge graph 1/16 generated.
Generating knowledge graph...


...

Knowledge graph generated.
Knowledge graph 14/16 generated.
Generating knowledge graph...


INFO | 2025-08-11 14:03:11,740 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:03:11,754 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:03:11,755 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:03:11,756 | Mappings processed in 0.540 seconds.
INFO | 2025-08-11 14:03:13,130 | Number of triples generated in total: 22243.


Knowledge graph generated.
Knowledge graph 15/16 generated.
Generating knowledge graph...


INFO | 2025-08-11 14:03:15,046 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:03:15,057 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:03:15,058 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:03:15,059 | Mappings processed in 0.538 seconds.
INFO | 2025-08-11 14:03:16,380 | Number of triples generated in total: 17015.


Knowledge graph generated.
Knowledge graph 16/16 generated.
Z3 literal set generated (1/16)
Z3 literal set generated (2/16)
Z3 literal set generated (3/16)
Z3 literal set generated (4/16)
Z3 literal set generated (5/16)
Z3 literal set generated (6/16)
Z3 literal set generated (7/16)
Z3 literal set generated (8/16)
Z3 literal set generated (9/16)
Z3 literal set generated (10/16)
Z3 literal set generated (11/16)
Z3 literal set generated (12/16)
Z3 literal set generated (13/16)
Z3 literal set generated (14/16)
Z3 literal set generated (15/16)
Z3 literal set generated (16/16)


In [14]:
len(z3_data)

359577

In [ ]:
len(mapper.data.data)

59801

In [16]:
str(z3_data[5008])

'hasResource(E_1174728, R_User_29)'

### Creating a simple ontology and performing reasoning
Some code here is taken from the opal library for testing purposes to demonstrate how reasoning works. The actual opal library usage will be much simpler

In [ ]:
# Define a fixed universe of constants and predicate symbols
x = Const('x', z3_literal.Ind)

# Mapping of logical operators
logical_ops = {
    'Or': z3.Or,
    'And': z3.And,
    'Not': z3.Not,
    'Implies': z3.Implies,
    'ForAll': z3.ForAll,
    'Exists': z3.Exists
}

predicate_map = SIGNATURE_REGISTER

# Recursive parser
def parse_expr(expr_str, var_ctx={}):
    expr_str = expr_str.strip()
    if expr_str.startswith('ForAll') or expr_str.startswith('Exists'):
        quantifier, inner = expr_str.split('(', 1)
        args = inner.rsplit(')', 1)[0]
        var_part, body_part = args.split(',', 1)
        var_name = var_part.strip()
        var = var_ctx.get(var_name, z3.Const(var_name, z3_literal.Ind))
        return logical_ops[quantifier](var, parse_expr(body_part.strip(), {**var_ctx, var_name: var}))
    elif '(' in expr_str:
        func_name, inner = expr_str.split('(', 1)
        inner_args = [parse_expr(arg.strip(), var_ctx) for arg in split_args(inner.rsplit(')', 1)[0])]
        if func_name in logical_ops:
            return logical_ops[func_name](*inner_args)
        elif func_name in predicate_map:
            return predicate_map[func_name](*inner_args)
        else:
            # otherwise, create a new z3 function symbol
            func = Function(func_name, *([z3_literal.Ind] * len(inner_args)), BoolSort())
            predicate_map[func_name] = func
            return predicate_map[func_name](*inner_args)
    else:
        # It's a variable name
        return var_ctx.get(expr_str, z3.Const(expr_str, z3.IntSort()))

# Helper to split arguments in a safe way
def split_args(s):
    args, depth, current = [], 0, ''
    for c in s:
        if c == ',' and depth == 0:
            args.append(current)
            current = ''
        else:
            if c == '(':
                depth += 1
            elif c == ')':
                depth -= 1
            current += c
    if current:
        args.append(current)
    return args

In [ ]:
axiom_str = 'ForAll(x, Not(ActivityOccurrence(x)))'
test_axiom = parse_expr(axiom_str)
print(test_axiom)

ForAll(x, Not(ActivityOccurrence(x)))


In [ ]:
# try the solver on basic axioms
z3.set_param(proof=True)
s = Solver()
s.set(unsat_core=True)
s.set(completion=True)
for i,literal in enumerate(z3_data):
  s.assert_and_track(literal.z3_expr, 'ind_'+str(i))
s.assert_and_track(test_axiom, 'axiom')

In [ ]:
# model generation / satisfiability check
if s.check() == sat:
    m = s.model()
    print("Theory is satisfiable -- model generated.")
    print(m)
else:
    print("Theory is unsatisfiable -- no model generated.")
    #pr = s.proof()
    #pr_sexpr = pr.sexpr()
    core = s.unsat_core()
    print(core)

Model is consistent.
[ind_231450 = True,
 ind_266179 = True,
 ind_262416 = True,
 ind_249560 = True,
 ind_136661 = True,
 ind_242604 = True,
 ind_340812 = True,
 E_383726 = Ind!val!3222,
 ind_247998 = True,
 ind_332867 = True,
 ind_177499 = True,
 ind_77003 = True,
 ind_350801 = True,
 ind_74937 = True,
 ind_187399 = True,
 ind_288339 = True,
 ind_651 = True,
 ind_324581 = True,
 E_1052929 = Ind!val!30392,
 ind_251959 = True,
 ind_112529 = True,
 E_667307 = Ind!val!10399,
 ind_249979 = True,
 ind_23714 = True,
 ind_216254 = True,
 ind_315838 = True,
 E_123675 = Ind!val!5886,
 ind_140653 = True,
 ind_118912 = True,
 ind_157310 = True,
 E_168459 = Ind!val!9547,
 ind_185260 = True,
 ind_293837 = True,
 ind_161476 = True,
 ind_212038 = True,
 ind_302014 = True,
 ind_362339 = True,
 ind_117731 = True,
 ind_346744 = True,
 ind_76577 = True,
 E_762664 = Ind!val!44378,
 ind_200572 = True,
 ind_359356 = True,
 E_192325 = Ind!val!1247,
 ind_169418 = True,
 ind_192775 = True,
 ind_323249 = True,


# Loading PSL Ontology
This is a work in progress

In [ ]:
from opal.logic.z3.z3_ontology import Z3Ontology
psl_core_smt2 = ''
# Initialize the ontology
ontology = Z3Ontology.from_smt2(psl_core_smt2)

Adding axiom: ont_t_0_type_partition
Parsing expression: ForAll([x], Or(activity(x), activity_occurrence(x), object_(x), event(x), transition(x))
Parsing expression: Or(activity(x), activity_occurrence(x), object_(x), event(x), transition(x)
Parsing expression: activity(x)
Parsing expression: x
Parsing expression: activity_occurrence(x)
Parsing expression: x
Parsing expression: object_(x)
Parsing expression: x
Parsing expression: event(x)
Parsing expression: x
Parsing expression: transition(x
Parsing expression: x
Adding axiom: ont_t_1_type_disjointness
Parsing expression: ForAll([x], And(
    Implies(activity(x), Not(Or(activity_occurrence(x), object_(x), event(x), transition(x)))),
    Implies(activity_occurrence(x), Not(Or(object_(x), activity(x), event(x), transition(x)))),
    Implies(object_(x), Not(Or(activity_occurrence(x), activity(x), event(x), transition(x)))),
    Implies(event(x), Not(Or(activity_occurrence(x), activity(x), object_(x), transition(x)))),
    Implies(transit

Z3Exception: Sort mismatch

In [9]:
#s = Solver()
ref_taxonomy_path = '../../opal/logic/z3/ontologies/ref/reference_taxonomy.smt2'
ref_z3 = parse_smt2_file(ref_taxonomy_path)